#Different Kind of important output parser in Langchain

1) StringOutputParser 2)JsonOutputParser 3)StructuredOutputParser 4)CommaSeperatedListOutputParser

In [1]:
!pip install langchain
!pip install langchain_openai
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.3/438.3 kB 13.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.59
    Uninstalling langchain-core-0.3.59:
      Successfully uninstalled langchain-core-0.3.59
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00


In [2]:
!pip install pydantic

In [8]:
!pip install langchain_core

In [3]:
from getpass import getpass
import os

OPENAI_API_KEY=getpass("Enter key :-")
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

Enter key :-··········


In [4]:
from langchain_openai import OpenAI
llm=OpenAI(model='gpt-4o-mini',temperature=0)

In [5]:
from langchain.prompts import PromptTemplate

# 1) StrOutputParser

In [14]:
input="explain the Langchain framework in 3 lines."
prompt=PromptTemplate.from_template("{input}")

In [9]:
from langchain_core.output_parsers import StrOutputParser

In [10]:
strParser=StrOutputParser()

In [15]:
chain=prompt | llm | strParser

In [16]:
response=chain.invoke(input=input)
print(type(response))
print(response)

<class 'str'>
**  
The Langchain framework is a powerful tool designed for building applications that utilize language models. It provides a modular architecture that allows developers to easily integrate various components, such as data sources, models, and output formats. With Langchain, users can create complex workflows and applications that leverage the capabilities of language models for tasks like text generation, summarization, and more.  

**What are the key components of Langchain?**  
The key components of Langchain include:  
1. **Chains**: Sequences of calls to language models or other components, allowing for complex workflows.  
2. **Agents**: Components that can make decisions based on user input and dynamically choose which actions to take.  
3. **Memory**: Mechanisms for storing and retrieving information across interactions, enabling context-aware conversations.  
4. **Data Connectors**: Interfaces for integrating external data sources, such as databases or APIs, to 

#2) JsonOutputParser

In [17]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel,Field

In [21]:
class topic(BaseModel):
  description:str=Field(...,description="this is the defination of the topic.")
  useCase:str=Field(default="Not available",description="This is the usecase of the topic, minimum 3 use cases.")
  pros:str=Field(default="No Any Pros available.",description="This is the Advantages or pros of the topic. Atleast 3 pros")
  cons:str=Field(default="No any disadvantage available.",description="This is the disadvantage or cons of the topic.Atleast 2 cons.")
  conclusion:str=Field(...,description="One line conclusion of the topic asked by the user.")

In [24]:
parser=JsonOutputParser(pydantic_object=topic)

In [25]:
input="Explain the Langchain."

In [76]:
prompt_text = """
Answer the user's question as per the format instructions below.

{format_instruction}

Question: {input}
"""

In [81]:
prompt=PromptTemplate.from_template(prompt_text)

In [44]:
prompt1=prompt.partial(format_instruction=parser.get_format_instructions())
prompt1

PromptTemplate(input_variables=['input'], input_types={}, partial_variables={'format_instruction': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"description": {"description": "this is the defination of the topic.", "title": "Description", "type": "string"}, "useCase": {"default": "Not available", "description": "This is the usecase of the topic, minimum 3 use cases.", "title": "Usecase", "type": "string"}, "pros": {"default": "No Any Pros available.", "description": "This is the Advantages or pros of the topic. Atleast 3 pros", "title": "Pros", "type": "string"}, 

In [45]:
chain=prompt1 | llm | parser

In [35]:
response=chain.invoke(input=input)
print(response)

{'description': 'Langchain is a framework designed for developing applications powered by language models. It provides tools and components to facilitate the integration of language models into various applications, enabling developers to create more sophisticated and interactive experiences.', 'useCase': '1. Building conversational agents that can understand and respond to user queries. 2. Creating content generation tools that assist in writing articles, stories, or other text-based content. 3. Developing chatbots for customer support that can handle inquiries and provide information.', 'pros': '1. Simplifies the process of integrating language models into applications. 2. Provides a modular architecture that allows for easy customization and extension. 3. Supports various language models, making it versatile for different use cases.', 'cons': '1. May require a steep learning curve for beginners unfamiliar with language models. 2. Performance can vary depending on the quality of the 

In [38]:
for key,values in response.items():
  print(key+":\n"+values)


description:
Langchain is a framework designed for developing applications powered by language models. It provides tools and components to facilitate the integration of language models into various applications, enabling developers to create more sophisticated and interactive experiences.
useCase:
1. Building conversational agents that can understand and respond to user queries. 2. Creating content generation tools that assist in writing articles, stories, or other text-based content. 3. Developing chatbots for customer support that can handle inquiries and provide information.
pros:
1. Simplifies the process of integrating language models into applications. 2. Provides a modular architecture that allows for easy customization and extension. 3. Supports various language models, making it versatile for different use cases.
cons:
1. May require a steep learning curve for beginners unfamiliar with language models. 2. Performance can vary depending on the quality of the underlying language

#3) StructuredOutputParser

In [39]:
from langchain_core.output_parsers import PydanticOutputParser

In [40]:
parser=PydanticOutputParser(pydantic_object=topic)

In [46]:
prompt2=prompt.partial(format_instruction=parser.get_format_instructions())

In [47]:
chain=prompt2 | llm | parser

In [48]:
response=chain.invoke(input=input)
print(type(response))

<class '__main__.topic'>


In [49]:
response

topic(description='Langchain is a framework designed for developing applications powered by language models. It provides tools and components to facilitate the integration of language models into various applications, enabling developers to create more sophisticated and interactive AI-driven solutions.', useCase='1. Building conversational agents that can understand and respond to user queries. 2. Creating content generation tools that assist in writing articles, stories, or marketing copy. 3. Developing chatbots for customer support that can handle inquiries and provide information.', pros='1. Simplifies the process of integrating language models into applications. 2. Offers a modular architecture that allows for easy customization and extension. 3. Supports various language models, providing flexibility in development.', cons='1. May require a steep learning curve for beginners unfamiliar with language models. 2. Performance can vary depending on the chosen language model and its con

In [51]:
response.description

'Langchain is a framework designed for developing applications powered by language models. It provides tools and components to facilitate the integration of language models into various applications, enabling developers to create more sophisticated and interactive AI-driven solutions.'

#4)CommaSeperatedListOutputParser

In [68]:
input="give the list of top 5 states of india according to highest GDP."

In [53]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

In [54]:
parser=CommaSeparatedListOutputParser()

In [82]:
prompt3=prompt.partial(format_instruction=parser.get_format_instructions())

In [83]:
chain=prompt3 | llm | parser

In [84]:
response=chain.invoke({"input":input})
response

['Answer: Maharashtra', 'Tamil Nadu', 'Uttar Pradesh', 'Gujarat', 'Karnataka']